## Fidelities with Common Randomized Measurements

With common randomized measurements [Vermersch, al, PRXQ 2024](https://doi.org/10.1103/PRXQuantum.5.010352), a classical approximation $\sigma$ of an experimental state $\rho$ is used to facilitate the measurement of physical properties. 

In this tutorial, we will illustrate in the context of the measurement of the fidelity $\mathcal{F}=\mathrm{Tr}(\rho \sigma)$, where $\sigma=\ket{\psi}\bra{\psi}$.

Note that in this example,  $\sigma$ is at the same time the observable we are interested in, and the classical approximation we used for building common randomized measurements.

In [1]:
using RandomMeas

ERROR: LoadError: PCRE compilation error: unknown property after \P or \p at offset 283
Stacktrace:
  [1] error(s::String)
    @ Base ./error.jl:35
  [2] compile(pattern::String, options::UInt32)
    @ Base.PCRE ./pcre.jl:165
  [3] compile(regex::Regex)
    @ Base ./regex.jl:80
  [4] Regex(pattern::String, compile_options::UInt32, match_options::UInt32)
    @ Base ./regex.jl:40
  [5] Regex(pattern::String)
    @ Base ./regex.jl:68
  [6] var"@r_str"(__source__::LineNumberNode, __module__::Module, pattern::Any, flags::Vararg{Any})
    @ Base ./regex.jl:123
  [7] include(mod::Module, _path::String)
    @ Base ./Base.jl:495
  [8] include(x::String)
    @ RandomMeas ~/Andreas_Arbeit/Projects/RandomMeas.jl/src/RandomMeas.jl:59
  [9] top-level scope
    @ ~/Andreas_Arbeit/Projects/RandomMeas.jl/src/RandomMeas.jl:103
 [10] include
    @ ./Base.jl:495 [inlined]
 [11] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::V

ErrorException: Failed to precompile RandomMeas [a9f96d1f-ed6a-4404-822f-5927c777f274] to "/Users/aelben/.julia/compiled/v1.10/RandomMeas/jl_mTQQAI".

We begin by considering $\sigma=\ket{\psi}\bra{\psi}$ ($\rho$) as the output of an ideal (noisy, respectively) $2$-qubit quantum circuit of various depths $d$

In [2]:
N = 4
depth = 5
cutoff = 1e-4
ξ = siteinds("Qubit", N)
ψ,ρ,F = MPS[], MPO[], Float64[]

#initial states
push!(ψ,MPS(ξ, ["Dn" for n in 1:N]));
push!(ρ,outer(ψ[1]',ψ[1]));
push!(F,1.)

#random circuit and noise parameters
circuit = random_circuit(ξ,depth)
p = 0.1*rand(N)
@showprogress dt=1 for d in 1:depth
            push!(ψ,apply(circuit,ψ[d];cutoff=cutoff))
            ρt = apply(circuit,ρ[d],apply_dag=true;cutoff=cutoff)
            #each random circuit layer is followed by local depolarization with probabilities p 
            push!(ρ,apply_depo_channel(ρt,p));  

            push!(F,real(inner(ψ[d+1]',ρ[d+1],ψ[d+1])))
end


UndefVarError: UndefVarError: `siteinds` not defined

We first perform randomized measurement on the experimental state, and store the unitaries $u$ that we used

In [3]:
#Perform Randomized measurements
NU=200
NM=2000

measurement_group = Vector{MeasurementGroup}(undef,depth+1)
@showprogress dt=1 for d in 1:depth+1
    measurement_group[d] = MeasurementGroup(ρ[d],NU,NM;mode="dense");
end

UndefVarError: UndefVarError: `MeasurementGroup` not defined

From randomized measurement, we know that we can build estimate the fidelity $\mathcal{F}$ as $\mathcal{F}_e=\mathrm{tr}(\sigma \hat{\rho})$ with the shadows $\hat{\rho}$. 

In [4]:
F_e = zeros(Float64,depth+1)
for d in 1:depth+1
    shadow_d = get_dense_shadows(measurement_group[d])
    F_e[d] = real(get_expect_shadow(outer(ψ[d]',ψ[d]),shadow_d))
end

UndefVarError: UndefVarError: `measurement_group` not defined

CRM provide a more efficient estimator $\mathcal{F}_{CRM}=\mathrm{tr}(\sigma \hat{\rho})-\mathrm{tr}(\sigma \hat{\sigma})+1$ based on  using the simulated shadow $\hat{\sigma}$, which is generated numerically from the Born probabilities $P_0(s)=\langle s|u\sigma u^\dag|s\rangle$.

In [5]:
F_CRM = copy(F_e).+1

for d in 1:depth+1
    for r in 1:NU
        data = measurement_group[d].measurements[r]
        P = MeasurementProbability(ψ[d],data.measurement_setting)
        shadow_sigma = DenseShadow(P)
        F_CRM[d] -= real(get_expect_shadow(outer(ψ[d]',ψ[d]),shadow_sigma))/NU
    end
end

UndefVarError: UndefVarError: `measurement_group` not defined

In [6]:
using Plots
Plots.plot(1:depth+1,F,label="Fidelity")
plot!(1:depth+1,F_e,label="Estimated")
plot!(1:depth+1,F_CRM,label="CRM estimated")
xlabel!("depth")
ylabel!("Fidelity")

UndefVarError: UndefVarError: `F` not defined